In [1]:
import os
import sys
import numpy as np
import pandas as pd

os.getcwd()
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(f'{parent_dir}/dsnp_transform')
import dsnpTransform as dsnpT


In [2]:
hra_with_broker_wo_iha = pd.read_pickle(r"Z:\Projects\COE\DSNP\dsnp_data_storage\monthly_flat_files\2024-05-31\mbr_hra_2024_with_broker_without_iha.pkl")
#pull_hedis = pd.read_pickle(r"Z:\Projects\COE\DSNP\dsnp_data_storage\monthly_flat_files\2024-05-31\pull_hedis.pkl")

In [3]:
# import pandas as pd
mbr_hra = pd.read_pickle(r"Z:\Projects\COE\DSNP\Ting\HRA\Result\20240505\mbr_hra_2024_with_broker_without_iha_revised.pkl")


def identify_long_gap(member_level_hra_completions):
    """
    This function computes flags for long-gap and past due members, as well as the number of HRAs completed in-year, the past 366 days, and the number of HRAs required to be 'caught up' from a compliance perspective.
    The input file structure is assumed to be the same as mbr_hra_2024_with_broker_without_iha, meaning the following fields are required:
        - HRA_Completion_Date_i, i in 1,...,8
        - I_eligible (Note: NO _final suffix - function currently DOES NOT WORK with IHA data. As a #TODO, we could strip the _final suffixes in the df after being pulled in to generate the same fields)
        - I_completed
        - I_duedate_max
        - R_eligible
        - R_completed
        - R_duedate_max_alt1 (another #TODO could be to strip _alt1 as a suffix in the initial setup of the function)
    It returns a dataframe of the same structure with the following fields added:
        - most_recent_hra_completion_date, second_most_recent_hra_completion_date
        - long_gap_flag: uses the clinical definition, ie, no HRA completions since start of prior year with an effective date before prior year start
        - number_of_hras_in_past_366
        - number_of_hras_since_year_start: year_start is current year
        - 


    """
    from datetime import date, timedelta
    import datetime as datetime
    from snp_query_box import DsnpHelperFunction
    import numpy as np
    import pandas as pd
    today = datetime.datetime.today()
    prior_year_start = np.datetime64(DsnpHelperFunction.first_date_of_year(date.today() - timedelta(days=365))) 
    current_year_start = np.datetime64(DsnpHelperFunction.first_date_of_year(date.today()))
    last_day_of_current_month = np.datetime64(DsnpHelperFunction.last_date_of_month(date.today()))
    last_day_of_prior_month = np.datetime64(DsnpHelperFunction.last_date_of_last_month(today))
    last_day_of_current_year = np.datetime64(DsnpHelperFunction.last_date_of_year(date.today()))
    member_level_hra_completions = member_level_hra_completions.copy()
    member_level_hra_completions[['second_most_recent_hra_completion_date','most_recent_hra_completion_date']] = np.sort(member_level_hra_completions[member_level_hra_completions.columns[member_level_hra_completions.columns.str.startswith('HRA_Completion_Date_')].tolist()].apply(pd.to_datetime, errors='coerce').fillna(np.datetime64('1970-01-01')))[:,-2:]
    member_level_hra_completions['long_gap_flag'] = np.where(((member_level_hra_completions['EFF_Date'] <= prior_year_start) & 
        ((member_level_hra_completions['HRA_Completion_Date_1'] < prior_year_start) |
        (member_level_hra_completions['HRA_Completion_Date_1'] >= current_year_start))), 1, 0)
    member_level_hra_completions['number_of_hras_in_past_366'] = member_level_hra_completions[member_level_hra_completions.columns[member_level_hra_completions.columns.str.startswith('HRA_Completion_Date_')].tolist()].apply(pd.to_datetime, errors='coerce').fillna(np.datetime64('1970-01-01')).gt(pd.to_datetime(np.datetime64('today')) - timedelta(days=366)).sum(axis=1)
    member_level_hra_completions['number_of_hras_since_year_start'] = member_level_hra_completions[member_level_hra_completions.columns[member_level_hra_completions.columns.str.startswith('HRA_Completion_Date_')].tolist()].apply(pd.to_datetime, errors='coerce').fillna(np.datetime64('1970-01-01')).gt(pd.to_datetime(np.datetime64(current_year_start))).sum(axis=1)

    member_level_hra_completions['past_due_flag'] = np.where((member_level_hra_completions['number_of_hras_in_past_366']) < 1 & (member_level_hra_completions['I_eligible'] == 0), 1, 0)
    member_level_hra_completions['past_due_still_in_play_flag'] = np.where(member_level_hra_completions['past_due_flag'] == 1, 1, 0)

    member_level_hra_completions['past_due_flag'] = np.where(((member_level_hra_completions['most_recent_hra_completion_date'] - timedelta(days=365) > member_level_hra_completions['second_most_recent_hra_completion_date']) & 
                                        (((member_level_hra_completions['most_recent_hra_completion_date'] > member_level_hra_completions['R_duedate_max_alt1']) & 
                                            (member_level_hra_completions['R_eligible']==1) & 
                                            (member_level_hra_completions['R_completed']==0))  |
                                            (((member_level_hra_completions['most_recent_hra_completion_date'] >= member_level_hra_completions['I_duedate_max']))) & 
                                            (member_level_hra_completions['I_eligible']==1) & 
                                            (member_level_hra_completions['I_completed']==0))) | 
                                            ((member_level_hra_completions['most_recent_hra_completion_date'] == np.datetime64('1970-01-01')) & 
                                            (member_level_hra_completions['I_duedate_max'] <= last_day_of_prior_month) &
                                            (member_level_hra_completions['I_eligible']==1) & 
                                            (member_level_hra_completions['I_completed']==0)), 1, member_level_hra_completions['past_due_flag'])
    
    member_level_hra_completions['past_due_still_in_play_flag'] = np.where((member_level_hra_completions['past_due_still_in_play_flag']==1) & (member_level_hra_completions['most_recent_hra_completion_date'] - timedelta(days=366) > member_level_hra_completions['second_most_recent_hra_completion_date']), 
                                                                           1, member_level_hra_completions['past_due_still_in_play_flag'])

    member_level_hra_completions['catch_up_hra_req'] = np.where((member_level_hra_completions['long_gap_flag']==1) | (member_level_hra_completions['past_due_flag']==1), 2-member_level_hra_completions['number_of_hras_since_year_start'],0)
 
    member_level_hra_completions['catch_up_due_date'] = np.where(member_level_hra_completions['catch_up_hra_req']==2,
                                                                 last_day_of_current_month,
                                                                 last_day_of_current_year)
    member_level_hra_completions['next_hra_due_date'] = np.where(
        (member_level_hra_completions['I_eligible'] == 1) & (member_level_hra_completions['I_completed'] == 0) & 
        (member_level_hra_completions['I_duedate_max'] >= last_day_of_prior_month), member_level_hra_completions['I_duedate_max'],
        np.where((member_level_hra_completions['R_eligible'] == 1) & ((member_level_hra_completions['past_due_flag']==0)) & 
                 (member_level_hra_completions['R_duedate_max_alt1'] >= last_day_of_prior_month), member_level_hra_completions['R_duedate_max_alt1'],
        member_level_hra_completions['catch_up_due_date']))


    member_level_hra_completions['next_hra_due_date'] = np.where(
    (member_level_hra_completions['I_completed'] == 1) | 
    (member_level_hra_completions['R_completed'] == 1), member_level_hra_completions['most_recent_hra_completion_date'] + timedelta(365),
    member_level_hra_completions['next_hra_due_date'])    

    member_level_hra_completions['next_hra_due_date'] = np.where((member_level_hra_completions['R_duedate_max_alt1'] <= member_level_hra_completions['next_hra_due_date']) & 
                                                                (member_level_hra_completions['R_duedate_max_alt1'] >= last_day_of_prior_month) & 
                                                                (member_level_hra_completions['R_eligible']==1) & 
                                                                (member_level_hra_completions['R_completed']==0), 
                                                                member_level_hra_completions['R_duedate_max_alt1'], 
                                                                member_level_hra_completions['next_hra_due_date'])
    
    member_level_hra_completions.loc[((member_level_hra_completions['I_eligible'] == 0) & (member_level_hra_completions['R_eligible'] == 0)) | 
                                     (member_level_hra_completions['Term_Date'] < last_day_of_prior_month),  'next_hra_due_date'] = pd.NaT    


    return member_level_hra_completions


hra_with_broker_wo_iha = identify_long_gap(hra_with_broker_wo_iha)

# test.loc[(test['I_eligible']==1) & (test['past_due_flag']==1),['EFF_Date','past_due_flag','long_gap_flag','catch_up_hra_req','most_recent_hra_completion_date','second_most_recent_hra_completion_date','I_eligible','I_completed','I_duedate_max','R_eligible','R_completed','R_duedate_max_alt1','catch_up_due_date','next_hra_due_date']].sort_values('catch_up_hra_req')


In [4]:
from datetime import date, timedelta
import datetime as datetime
from snp_query_box import DsnpHelperFunction
import numpy as np
import pandas as pd
today = datetime.datetime.today()

last_day_of_prior_month = np.datetime64(DsnpHelperFunction.last_date_of_last_month(today))
hra_flag_df = dsnpT.get_hra_flag(hra_with_broker_wo_iha, np.datetime64('2024-05-31'))
hra_flag_df.head()
hra_flag_df['next_hra_due_date_alt'] = np.where((hra_flag_df['R_duedate_max_alt1'] <= hra_flag_df['next_hra_due_date']) & 
                                                            (hra_flag_df['R_duedate_max_alt1'] >= last_day_of_prior_month) & 
                                                            (hra_flag_df['R_eligible']==1) & 
                                                            (hra_flag_df['R_completed']==0), 
                                                            hra_flag_df['R_duedate_max_alt1'], 
                                                            hra_flag_df['next_hra_due_date'])

hra_flag_df.loc[(hra_flag_df['R_duedate_max_alt1'] <= hra_flag_df['next_hra_due_date']) & 
                                                            (hra_flag_df['R_duedate_max_alt1'] >= last_day_of_prior_month) & 
                                                            (hra_flag_df['R_eligible']==1) & 
                                                            (hra_flag_df['next_hra_due_date']!=hra_flag_df['next_hra_due_date_alt']) ,['Member_ID','EFF_Date','next_hra_due_date_alt','next_hra_due_date','R_duedate_max_alt1','past_due_flag','R_eligible','R_completed','I_eligible','I_completed','most_recent_hra_completion_date','second_most_recent_hra_completion_date']]

,Member_ID,EFF_Date,next_hra_due_date_alt,next_hra_due_date,R_duedate_max_alt1,past_due_flag,R_eligible,R_completed,I_eligible,I_completed,most_recent_hra_completion_date,second_most_recent_hra_completion_date


In [5]:
hra_flag_df.loc[(hra_flag_df['R_completed']==0) & (hra_flag_df['R_eligible']==0) & (hra_flag_df['I_eligible']==1) & (hra_flag_df['I_completed']==1),['Term_Date','most_recent_hra_completion_date','second_most_recent_hra_completion_date','I_duedate_max','catch_up_hra_req','R_duedate_max_alt1','I_eligible','I_completed','R_eligible','R_completed','HRA_flag','next_hra_due_date']]

,Term_Date,most_recent_hra_completion_date,second_most_recent_hra_completion_date,I_duedate_max,catch_up_hra_req,R_duedate_max_alt1,I_eligible,I_completed,R_eligible,R_completed,HRA_flag,next_hra_due_date
2,2024-02-29,2024-03-01,2024-01-10,2024-03-31,0,2024-12-31,1,1,0,0,HRA Compliant,NaT
3,2099-12-31,2024-03-01,2024-01-10,2024-06-30,0,2025-04-01,1,1,0,0,HRA Compliant,2025-03-01
12,2024-02-29,2024-02-22,1970-01-01,2024-05-01,0,2025-01-31,1,1,0,0,HRA Compliant,NaT
14,2099-12-31,2024-03-27,1970-01-01,2024-06-30,0,2025-04-01,1,1,0,0,HRA Compliant,2025-03-27
16,2024-02-29,2024-02-23,1970-01-01,2024-03-31,0,2024-12-31,1,1,0,0,HRA Compliant,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
588867,2099-12-31,2024-03-23,1970-01-01,2024-03-31,0,2024-12-31,1,1,0,0,HRA Compliant,2025-03-23
588868,2024-03-31,2024-02-06,1970-01-01,2024-05-30,0,2025-03-01,1,1,0,0,HRA Compliant,NaT
588869,2099-12-31,2024-02-11,1970-01-01,2024-05-01,0,2025-01-31,1,1,0,0,HRA Compliant,2025-02-10
588870,2024-02-29,2024-01-26,1970-01-01,2024-05-01,0,2025-01-31,1,1,0,0,HRA Compliant,NaT


In [6]:
hra_flag_df.loc[hra_flag_df['Member_ID'].isin(['101465645900', '101527150400']),['Member_ID','Contract_Number','EFF_Date','Term_Date','I_completed_date','R_completed_date']]

,Member_ID,Contract_Number,EFF_Date,Term_Date,I_completed_date,R_completed_date
75931,101527150400,H3239,2024-03-01,2024-03-31,2023-12-05,2024-03-18
75932,101527150400,H3239,2024-05-01,2024-05-31,2024-02-19,1970-01-01
92377,101465645900,H8597,2024-03-01,2024-03-31,2023-12-05,1970-01-01
92378,101465645900,H8597,2024-05-01,2024-05-31,1970-01-01,1970-01-01


In [7]:
hra_flag_df.loc[(hra_flag_df['Term_Date'] <= np.datetime64('2024-05-31')) & (hra_flag_df['past_due_flag']==1),['Term_Date','most_recent_hra_completion_date','second_most_recent_hra_completion_date','I_duedate_max','catch_up_hra_req','R_duedate_max_alt1','I_eligible','I_completed','R_eligible','R_completed','HRA_flag','next_hra_due_date']].sort_values('next_hra_due_date')

,Term_Date,most_recent_hra_completion_date,second_most_recent_hra_completion_date,I_duedate_max,catch_up_hra_req,R_duedate_max_alt1,I_eligible,I_completed,R_eligible,R_completed,HRA_flag,next_hra_due_date
32,2024-02-29,1970-01-01,1970-01-01,2024-05-01,2,2025-01-31,0,0,0,0,HRA Past Due - Member Termed,NaT
38,2024-02-29,1970-01-01,1970-01-01,2024-02-29,2,2024-11-30,1,0,0,0,HRA Past Due - Member Termed,NaT
58,2024-04-30,1970-01-01,1970-01-01,2024-05-30,2,2025-03-01,0,0,0,0,HRA Past Due - Member Termed,NaT
92,2024-03-31,1970-01-01,1970-01-01,2024-05-01,2,2025-01-31,0,0,0,0,HRA Past Due - Member Termed,NaT
96,2024-01-31,1970-01-01,1970-01-01,2024-03-31,2,2024-12-31,0,0,0,0,HRA Past Due - Member Termed,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
588766,2024-03-31,1970-01-01,1970-01-01,2024-05-30,2,2025-03-01,0,0,0,0,HRA Past Due - Member Termed,NaT
588784,2024-01-31,2023-05-02,1970-01-01,2020-12-30,2,2024-05-01,0,0,0,0,HRA Past Due - Member Termed,NaT
588842,2024-01-31,1970-01-01,1970-01-01,2024-03-31,2,2024-12-31,0,0,0,0,HRA Past Due - Member Termed,NaT
588853,2024-03-31,2022-09-07,1970-01-01,2021-05-02,2,2023-09-07,0,0,1,0,HRA Past Due - Member Termed,NaT


In [8]:
hra_flag_df.loc[hra_flag_df['HRA_flag']=='HRA Not Eligible',['Term_Date','next_hra_due_date']]

,Term_Date,next_hra_due_date
10,2024-01-31,NaT
84,2024-01-31,NaT
118,2024-01-31,NaT
152,2024-01-31,NaT
174,2024-05-31,NaT
...,...,...
588434,2024-06-30,NaT
588612,2024-01-31,NaT
588748,2024-04-30,NaT
588769,2024-03-31,NaT


In [9]:
hra_flag_df.loc[hra_flag_df['HRA_flag']=='Undefined',['I_eligible','I_completed','R_eligible','R_completed','long_gap_flag','past_due_flag']].value_counts(['I_eligible','I_completed','R_eligible','R_completed','long_gap_flag','past_due_flag'])

I_eligible  I_completed  R_eligible  R_completed  long_gap_flag  past_due_flag
0           0            1           0            1              0                10
                                                  0              0                 3
1           0            0           0            0              0                 3
Name: count, dtype: int64

In [13]:
hra_flag_df.loc[(hra_flag_df['HRA_flag']=='Undefined'),['Member_ID','EFF_Date','I_eligible','I_completed','R_eligible','R_completed','Term_Date','past_due_flag','I_duedate_max','R_duedate_max_alt1','most_recent_hra_completion_date','second_most_recent_hra_completion_date']]

,Member_ID,EFF_Date,I_eligible,I_completed,R_eligible,R_completed,Term_Date,past_due_flag,I_duedate_max,R_duedate_max_alt1,most_recent_hra_completion_date,second_most_recent_hra_completion_date
169,101121096100,2022-01-01,0,0,1,0,2024-03-31,0,2022-04-01,2024-02-10,2024-05-21,2024-02-23
3658,101463287100,2021-07-01,0,0,1,0,2024-01-31,0,2021-09-29,2022-07-08,2024-05-15,2024-01-30
7503,101467462200,2022-11-01,0,0,1,0,2024-01-31,0,2023-01-30,2023-12-28,2024-05-13,2024-04-29
11057,101647924000,2023-01-01,0,0,1,0,2024-01-31,0,2023-04-01,2023-12-06,2024-04-05,2024-03-28
11296,101141353600,2024-01-01,1,0,0,0,2024-03-31,0,2024-03-31,2024-12-31,2024-05-22,2024-04-04
13879,101590956300,2023-01-01,0,0,1,0,2024-01-31,0,2023-04-01,2023-11-09,2024-03-07,2024-01-24
25106,101534828400,2024-01-01,1,0,0,0,2024-03-31,0,2024-03-31,2024-12-31,2024-05-29,2024-05-22
32646,101530903800,2023-01-01,0,0,1,0,2024-03-31,0,2023-04-01,2023-11-02,2024-05-14,2024-01-12
38684,101619239600,2023-01-01,0,0,1,0,2024-01-31,0,2023-04-01,2023-12-01,2024-05-07,2024-04-16
47432,101678481700,2023-02-01,0,0,1,0,2024-02-29,0,2023-05-02,2024-01-04,2024-04-29,2024-03-15


In [17]:
hra_flag_df.loc[hra_flag_df['Member_ID']=='101121096100',['EFF_Date','Term_Date','Contract_Number','HRA_Completion_Date_1','HRA_Completion_Date_2','HRA_Completion_Date_3','HRA_Completion_Date_4']]

,EFF_Date,Term_Date,Contract_Number,HRA_Completion_Date_1,HRA_Completion_Date_2,HRA_Completion_Date_3,HRA_Completion_Date_4
169,2022-01-01,2024-03-31,H1609,2023-02-10,2024-02-23,2024-05-21,NaT
170,2024-06-01,2099-12-31,H1609,NaT,2024-02-23,2024-05-21,NaT


In [11]:
hra_flag_df['HRA_flag'].value_counts()

HRA_flag
HRA Coming Due                                217932
HRA Compliant                                 212212
HRA Past Due                                   71237
HRA Past Due - Member Termed                   45724
HRA Long Gap - Need 2 HRAs                     17108
HRA Completed After Due Date                    9516
HRA Not Eligible                                9488
HRA Long Gap - Need 1 More HRA                  5607
Missed Initial HRA, Reassessment Compliant        45
Undefined                                         16
Name: count, dtype: int64

In [ ]:
hra_flag_df.loc[(hra_flag_df['HRA_flag']=='Undefined'),['I_eligible','I_completed','R_eligible','R_completed','Term_Date','past_due_flag','I_duedate_max','R_duedate_max_alt1','most_recent_hra_completion_date','second_most_recent_hra_completion_date']]